In [ ]:
import keras
import tensorflow as tf
import keras.layers as layers
from keras.preprocessing.image import load_img
from keras.preprocessing.image import array_to_img
import numpy as np
from math import floor, ceil
from tensorflow.python.ops import math_ops
from tensorflow import math, random, shape
import os
from keras.losses import MeanSquaredError, BinaryCrossentropy
from keras.optimizers import Nadam, SGD, Adam, Adamax
from keras.activations import sigmoid
from tensorflow import convert_to_tensor as tens
from keras import backend as K
from cv2 import getGaborKernel as Gabor
from functools import reduce
from matplotlib import pyplot as plt
from math import sqrt
import itertools
import re
from random import shuffle, seed
from tensorflow.keras.utils import Sequence
from keras.constraints import NonNeg
from keras.regularizers import l1,l2,l1_l2
from keras.initializers import RandomNormal
import pickle

In [ ]:
N_EXC = 639
BATCH_SIZE = 16
RESOLUTION = 8
EXCITATORY_SYNAPSES_WANTED = 8
INHIBITORY_SYNAPSES_WANTED = 6

# Transform data to dataset

In [ ]:
directory = '/kaggle/input/cat-and-dog/'
LABELS = ['Cat', 'Dog']

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory+'training_set/training_set',
    labels='inferred',
    color_mode='grayscale',
    validation_split=0.2,
    subset="training",
    seed=1337,
    batch_size=BATCH_SIZE,
)
valid_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory+'training_set/training_set',
    labels='inferred',
    color_mode='grayscale',
    validation_split=0.2,
    subset="validation",
    seed=1337,
    batch_size=BATCH_SIZE,
)

test_ds = keras.preprocessing.image_dataset_from_directory(directory+'test_set/test_set', labels='inferred', color_mode='grayscale', batch_size=BATCH_SIZE)

show some photos

In [ ]:
plt.figure(figsize=(10, 10))
for i, (images, labels) in enumerate(train_ds.take(9)):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[0].numpy()[:,:,0], cmap='gray')
    plt.title(LABELS[int(labels[0])])
    plt.axis("off")

# Use David Beniaguev's (selfishgene) trained L5PC model

In [ ]:
class Module:
    def __init__(self, path, module_name, name):
        self.name = name
        self.module = self.create_module(path, module_name)
    
    def create_module(self, path, module_name):
        models_folder  = os.path.join(path, 'Models')
        model_filename  = os.path.join(models_folder, module_name)
        old_model = keras.models.load_model(model_filename)
        inp = keras.Input(shape=old_model.layers[0].input.shape[1:])
        x = old_model.layers[1](inp)
        for layer in old_model.layers[2:-3]:
            x = layer(x)
        module = keras.Model(inp, [layer(x) for layer in old_model.layers[-3:-1]])
#         L5PC_model.compile(optimizer=Nadam(lr=0.0001), loss='binary_crossentropy', loss_weights=[1.])

        for layer in module.layers:
            layer.trainable = False

        module.summary()
        return module   
    def __call__(self, x):
        return self.module(x)

In [ ]:
multiple_neurons = False
dataset_folder = '/kaggle/input/single-neurons-as-deep-nets-nmda-test-data'

In [ ]:
if multiple_neurons:
    neurons = [Module(dataset_folder, "NMDA_TCN__DWT_8_224_217__model.h5", "Module_8_layers"), 
           Module(dataset_folder, "NMDA_TCN__DWT_7_292_169__model.h5", "Module_7_layers"), 
           Module(dataset_folder, "NMDA_TCN__DWT_9_256_241__model.h5", "Module_9_layers")]
else:
    neurons = Module(dataset_folder, "NMDA_TCN__DWT_7_128_153__model.h5", "module_7_layers")

# Some custom layers and funcs to use

In [ ]:
class ToBoolLayer(keras.layers.Layer):
  def __init__(self, threshold=0.5, mult=15, use_sigmoid=True, use_special_sigmoid=None, name="ToBoolLayer"):
      super().__init__(name=name)
      self.use_sigmoid = use_sigmoid
      self.special_sigmoid = use_special_sigmoid
      self.threshold = threshold
      self.mult = mult

  def build(self, shape):
    if self.special_sigmoid is not None:
      if not len(self.special_sigmoid) == 2:
        raise Exception("Special sigmoid should be in format (pos_mult, neg_mult).")
      else:
        self.sigmoid = SigmoidThreshold(*self.special_sigmoid, self.threshold)
    elif self.use_sigmoid:
      self.sigmoid = SigmoidThresholdEasier(threshold=self.threshold, mult=self.mult)
    
  def call(self, inputs, training=None):
    if training is False:
      inputs = inputs - self.threshold
      inputs = math_ops.ceil(inputs)
    elif self.use_sigmoid:
      inputs = self.sigmoid(inputs)
    return inputs

In [ ]:
def SigmoidThresholdEasier(mult=1, threshold=0.5):
  """returns a sigmoid function [0,1]->[0,1] with the center at threshold given, and slope multified"""
  def sigmoid_threshold(x):
      return 1/(1+math.exp(mult*(threshold-x)))
  return sigmoid_threshold

how the new sigmoid looks like

In [ ]:
x = np.arange(0,1,0.01)
threshold = 0.9
mult = 50
plt.plot(x, SigmoidThresholdEasier(mult, threshold)(x))
plt.ylim(0,1)
plt.xlim(0,1)
plt.axhline(0.5, color='r', linestyle='--')
plt.axvline(threshold, color='g', linestyle='--')
plt.show()

In [ ]:
class ToNeuronInput(keras.layers.Layer):
    """flattens the temporal dimensions, orders by new_order and pads with 0's to fill"""
    def __init__(self, full, padding=0, new_order=None, name="NeuronInput", part=200, batch_size=BATCH_SIZE):
        super().__init__(name=name)
        self.full = full
        self.new_order = None
        self.zero_padding = isinstance(padding, int) and padding == 0
        self.padding = padding
        self.part = part
        self.batch_size = batch_size
    
    def build(self, shape):
        self.ms = shape[1]*shape[2]
        self.times = self.part // self.ms
        self.padding_amount = self.full-self.ms*self.times
    
    def call(self, inputs, padding):
                
        new_inp = layers.Reshape((self.ms, inputs.shape[-1]))(inputs)
        if self.new_order is not None:
            new_inp = gather(new_inp, self.new_order, axis=-2)
        if self.times > 1:
            new_inp = layers.Concatenate(axis=1)([new_inp for _ in range(self.times)])
        if self.zero_padding and False:
            new_inp = layers.ZeroPadding1D(padding=(self.full - self.ms*self.times, 0))(new_inp)
        else:
            padding = tf.reshape(padding, (padding.shape[0]*padding.shape[1]*padding.shape[2], padding.shape[-1]))
            starting_time = layers.Lambda(lambda x: self.ranInt(x))(padding)
            padding = padding[np.newaxis, starting_time:starting_time+self.padding_amount]
            new_inp = layers.Concatenate(axis=-2)([tf.tile(padding, [tf.shape(new_inp)[0], 1, 1]), new_inp])
        return new_inp

    def ranInt(self, x):
        return K.random_uniform((1,), 0, 1024-self.padding_amount, dtype=tf.int32)[0]#.numpy()
    
    @tf.function
    def gather(x, ind):
        return tf.gather(x+0, ind)

In [ ]:
class SpikeProcessor(keras.layers.Layer):
    def __init__(self, nSynapses, start=None, end=None, name="SpikeProcessor"):
        super().__init__(name=name)
        self.nSynapses = nSynapses
        self.start = start
        self.end = end

    def build(self, shape):
        self.dims = len(shape)
        pass
    
    def call(self, inputs):
        if self.start is not None:
            if self.end is not None:
                inputs = inputs[:,:,:,self.start:self.end] if self.dims==4 else inputs[:,:,self.start:self.end]
            else:
                inputs = inputs[:,:,:,self.start:] if self.dims==4 else inputs[:,:,self.start:]
        elif self.end is not None:
            inputs = inputs[:,:,:,:self.end] if self.dims==4 else inputs[:,:,:self.end]
        preds_sum = math.reduce_sum(inputs, axis=-1, keepdims=True)
        return_value = preds_sum/self.nSynapses
        return layers.Flatten()(return_value)

In [ ]:
def MeanSquaredErrorSynapsesPerMS(batch_size=32):
  def mean_squared_error_synapses_per_ms(y_true, y_preds):
    squared_difference = tf.square(1-y_preds)
    mean = tf.reduce_mean(squared_difference, axis=-1)
    return mean
  return mean_squared_error_synapses_per_ms

In [ ]:
max_acceptable_spikes_per_ms = 3.0
max_acceptable_spikes_deviation = 20.0
activity_reg_constant = 0.2 * 0.0028


def pre_synaptic_spike_regularization(activation_map):
    # sum over all dendritic locations
    x = K.sum(activation_map, axis=2)

    # ask if it's above 'max_acceptable_spikes_per_ms'
    x = K.relu(x - max_acceptable_spikes_per_ms)

    # if above threshold, apply quadratic penelty
    x = K.square(x / max_acceptable_spikes_deviation)

    # average everything
    x = activity_reg_constant * K.mean(x)

    return x

# **Preprocessing**

Parameters for gabor filters and max pooling kernel sizes are from "Robust Object Recognition with Cortex-Like Mechanisms" (Serre et al.)

> # Gabor filters - Simple Cells

In [ ]:
n_filters = 64
n_orientations = 4

ksizes = [(i, i) for i in range(7,38, 2)]
thetas = [0 , (45 / 180) * np.pi, (90 / 180) * np.pi, (135 / 180) * np.pi]
gammas = [0.3] * 16
sigmas = [2.8, 3.6, 4.5, 5.4, 6.3, 7.3, 8.2, 9.2, 10.2, 11.3, 12.3, 13.4, 14.6, 15.8, 17., 18.2]
lambdas = [3.5, 4.6, 5.6, 6.8, 7.9, 9.1, 10.3, 11.5, 12.7, 14.1, 15.4, 16.8, 18.2, 19.7, 21.2, 22.8]

all_filters = [[(size, sigma, theta, lambd, gamma) for theta in thetas] 
               for size, gamma, sigma, lambd in zip(ksizes, gammas, sigmas, lambdas)]
all_filters = reduce(lambda x,y: x+y, all_filters, [])
reoredering_inds = [ 0,0+4,   1, 1+4,  2, 2+4,  3, 3+4,
                     8,8+4,   9, 9+4, 10,10+4, 11,11+4,
                    16,16+4, 17,17+4, 18,18+4, 19,19+4,
                    24,24+4, 25,25+4, 26,26+4, 27,27+4,
                    32,32+4, 33,33+4, 34,34+4, 35,35+4,
                    40,40+4, 41,41+4, 42,42+4, 43,43+4,
                    48,48+4, 49,49+4, 50,50+4, 51,51+4,
                    56,56+4, 57,57+4, 58,58+4, 59,59+4]

all_filters_reordered = [all_filters[k] for k in reoredering_inds]

Gabor initializer

In [ ]:
class GaborInitializer(tf.keras.initializers.Initializer):
    def __init__(self, size, sigma, theta, lambd, gamma):
        self.ksize = size
        self.sigma = sigma
        self.theta = theta
        self.lambd = lambd
        self.gamma = gamma

    def __call__(self, dtype=None):
        return tens(Gabor(self.ksize, self.sigma, self.theta, self.lambd, self.gamma))

    def get_config(self):  # To support serialization
        return {'ksize': self.ksize, 'sigma': self.sigma, 'theta': self.theta, 'lambda': self.lambd, 'gamma': self.gamma}

In [ ]:
max_filter_shape = all_filters_reordered[-1][0]
center_pixel_ind = int((max_filter_shape[0] - 1 ) / 2)

# place to store all filter activations
filters_matrix = np.zeros((max_filter_shape[0], max_filter_shape[1], len(all_filters)))

plt.figure(figsize=(18,22))
plt.subplots_adjust(left=0.04, right=0.96, bottom=0.04, top=0.96, hspace=0.25, wspace=0.1)
for i, filt in enumerate(all_filters_reordered):
    filter_size  = filt[0][0]
    oritentation = (filt[2] / np.pi ) * 180
    curr_sigma   = filt[1]
    curr_lambda  = filt[3]
    
    half_filter_size = int((filter_size -1 ) / 2)
    upper_left_start = center_pixel_ind - half_filter_size
    
    curr_small_filter = GaborInitializer(*filt)().numpy()
    curr_full_filter = np.zeros((max_filter_shape))
    curr_full_filter[upper_left_start:upper_left_start + filter_size, upper_left_start:upper_left_start + filter_size] = curr_small_filter
    
    # store the filter and the activations for later
    filters_matrix[:,:,i] = curr_full_filter
    
    plt.subplot(8,8,i+1);
    plt.title('%dx%d, $\Theta=%d$, \n$\sigma=%.1f$, $\lambda=%.1f$' %(filter_size, filter_size, oritentation, curr_sigma, curr_lambda))
    plt.imshow(curr_full_filter, cmap='gray')
    plt.axis("off")

Function to create Conv2D layer waith gabor filter

In [ ]:
def simple_cells_module(filters_matrix, strides=(1,1), input_shape=(256,256)):
    # input is a single channel gray scale image
    input_tensor = keras.Input(shape=(input_shape[0],input_shape[1],1))
    
    # initializer with predefined weights
    def gabor_filters_init(shape, dtype=None):
        return -filters_matrix[:,:,np.newaxis,:]

    num_filters = filters_matrix.shape[2]
    kernel_size = (filters_matrix.shape[0], filters_matrix.shape[1])

    # single conv2d layer with all weights
    conv_2d_layer = layers.Conv2D(num_filters, kernel_size, strides=strides, kernel_initializer=gabor_filters_init, padding='same')
    conv_2d_layer.trainable=False
    simple_cell_activations = conv_2d_layer(input_tensor)
    simple_cell_module = keras.Model(input_tensor, simple_cell_activations)
    
    return simple_cell_module

In [ ]:
max_pooling_size = list(range(8,23,2))
pooling_size_list = [[(s,s,2)] for s in max_pooling_size]
num_orientations = 4

In [ ]:
def pre_process_module(filter_matrix, pooling_size_list, strides=(1,1), num_orientations=4, input_shape=(256,256)):
    
    print('num gabor filters must be %d' %(sum([x[0][-1] for x in pooling_size_list]) * num_orientations))

    # input is a single channel gray scale image
    input_tensor = keras.Input(shape=(input_shape[0],input_shape[1],1))
    
    # calc simple cell activations
    simple_cells_activations = K.expand_dims(simple_cells_module(filters_matrix, strides=(1,1), input_shape=(256,256))(input_tensor), axis=-1)
    
    # add support for subsampling
    strides_to_use = (strides[0], strides[1], 2)

    # apply max pooling
    maxpool_3d_layers = []
    for k, pool_size in enumerate(pooling_size_list):
        start_ind = num_orientations * pool_size[0][-1] * k
        end_ind   = num_orientations * pool_size[0][-1] * (k + 1)
        curr_pool_input_slice  = simple_cells_activations[:,:,:,start_ind:end_ind]
        curr_pool_output_slice = layers.MaxPooling3D(pool_size=pool_size[0], strides=strides_to_use, padding='same')(curr_pool_input_slice)
        maxpool_3d_layers.append(curr_pool_output_slice)
    
    # squeeze the last dimention
    concatenated_pooled_layers = K.squeeze(layers.Concatenate(axis=-2)(maxpool_3d_layers), axis=-1)
    
    # wrap as module and return
    complex_cell_module = keras.Model(input_tensor, concatenated_pooled_layers)
    
    return complex_cell_module

In [ ]:
def preprocessor(filters_matrix, pooling_sizes, n_orientations=4, conv_strides=(1,1), max_pooling_strides=(1,1), input_shape=(256,256)):
    return pre_process_module(filters_matrix, pooling_sizes, strides=max_pooling_strides, num_orientations=n_orientations, input_shape=input_shape)


In [ ]:
processor = preprocessor(filters_matrix, pooling_size_list, max_pooling_strides=(RESOLUTION,RESOLUTION))

In [ ]:
plt.figure(figsize=(10, 10))
for _, (image, label) in enumerate(train_ds.take(1)):
    image = image[0].numpy()[:,:,0]
    label = int(label[0])
    processed = processor(image[np.newaxis,:,:,np.newaxis]).numpy()[0]
    for i in range(32):
        ax = plt.subplot(8, 4, i+1)
        plt.imshow(processed[:,:,i], cmap='gray')
        plt.title(LABELS[label])
        plt.axis("off")

# Start noise

In [ ]:
plt.figure(figsize=(5,20))
dct = {"Dog": [], "Cat": []}
for _, (image, label) in enumerate(train_ds.take(5)):
    for img, lbl in zip(image, label):
        dct[LABELS[lbl]].append(img[np.newaxis,])

# print(f"Dog images: {len(dct["Dog"])}\n Cat images: {len(dct["Cat"])}")

all_images = np.concatenate(dct["Dog"] + dct["Cat"], axis=0)
plt.subplot(4,1,1)
plt.title("Original Image")
plt.imshow(all_images[0,:,:,0], cmap="gray")

blocksize = 64

# Create blocks
shuffled_images = all_images.copy()
for j in range(0, all_images.shape[2], blocksize):
    for i in range(0, all_images.shape[1], blocksize):
        indxs = np.random.permutation(all_images.shape[0]).tolist()
        for orig,new in zip(indxs, range(all_images.shape[0])):
            shuffled_images[orig,i:i+blocksize, j:j+blocksize] = all_images[new, i:i+blocksize, j:j+blocksize]

plt.subplot(4,1,2)
plt.title("Hybrid Image")
plt.imshow(shuffled_images[0,:,:,0], cmap="gray")
processed_images = processor(shuffled_images)

plt.subplot(4,1,3)
plt.title("Processed Hybrid Image (Channel 0)")
plt.imshow(processed_images[0,:,:,0], cmap="gray")

image_list = [processed_images[i] for i in range(processed_images.shape[0])]
PADDING = np.concatenate(image_list, axis=-2)[np.newaxis]

plt.subplot(4,1,4)
plt.title("Concatenated Processed Hybrid Image (Channel 0)")
plt.imshow(PADDING[0,:,:,0], cmap="gray")

plt.show()

# A little augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal"),
  layers.experimental.preprocessing.RandomRotation(0.1),
])

In [ ]:
plt.figure(figsize=(30, 10))
for _, (images, labels) in enumerate(train_ds.take(1)):
    augmented = data_augmentation(images)
    for i in range(9):
        plt.subplot(2, 9, i + 1)
        plt.imshow(images[i,:,:,0], cmap='gray')
        plt.title(LABELS[int(labels[i])])
        plt.axis("off")
        
        plt.subplot(2, 9, 9 + i + 1)
        plt.imshow(augmented[i,:,:,0], cmap='gray')
        plt.title("Augmented " + LABELS[int(labels[i])])
        plt.axis("off")

In [ ]:
def padding_module(use_sigmoid=True, pruning=True, dropout1=False, dropout2=False, 
                  sigmoid_threshold=0.9, sigmoid_mult=50, to_bool=True, 
                  padding=PADDING):
    
    def printModule():
        print("~*~ Visual Module ~*~")
        print(f"Image Dropout Rate: {dropout1}")
        print(f"Synapse Threshold: {sigmoid_threshold}")
        print(f"Synapse Training Sigmoid Multiplication: {sigmoid_mult}")
        print(f"Synapses Dropout Rate: {dropout2}")
    
    
    printModule()
    
    inp = keras.Input(shape=(256,256,1))
    
    padding = tf.Variable(lambda: padding, trainable=False)
    x = inp
    x = processor(x)
    if dropout1: x = layers.Dropout(dropout1)(x)
    conv_shape = (x.shape[-3], 1)
    x = Pad(padding)(x)
    x = layers.Conv2D(1278, conv_shape, strides=conv_shape, activity_regularizer=pre_synaptic_spike_regularization, name="WiringLayer")(x)
    x = layers.BatchNormalization(name="BatchNorm")(x)
    x = layers.Activation(sigmoid)(x)
    x = ToBoolLayer(threshold=sigmoid_threshold, use_sigmoid=use_sigmoid, mult=sigmoid_mult, name='preNeuronBool')(x)
    
    x = K.squeeze(x, axis=-3)

    if dropout2: x = layers.Dropout(dropout2)(x)
    x, v = L5PC_model(x)
    x = x[:, -198:, :]
#     x = L5PC_model(x)[:,-198:,:]   # run through david's model, take only the post-noise time (32 ms X 6 times)
    x = layers.MaxPooling1D(x.shape[-2], strides=x.shape[-2], name="MaxPooling")(x)
    output = layers.Flatten(name="nSpikes")(x)

    model = keras.Model(inp, [output, v])
    model.compile(optimizer=optimizer, loss={'nSpikes': MeanSquaredError()}, metrics={'nSpikes': keras.metrics.BinaryAccuracy()})
    return model

# Module

In [ ]:
def printModule(dropout1, dropout2, sigmoid_threshold, sigmoid_mult, threshold, exWanted, inWanted, qSynapse, augment, synLoss):
    print("~*~ Visual Module ~*~")
    if augment: print(f"Images are augmented (rotated by {augment})")
    print(f"Image Dropout Rate: {dropout1}")
    print(f"Synapse Threshold: {sigmoid_threshold}")
    print(f"Synapse Training Sigmoid Multiplication: {sigmoid_mult}")
    print(f"Synapses Wanted: Exc-{exWanted}; Inh-{inWanted}; Sum-{exWanted+inWanted}")
    print(f"Synapses Loss Rate: Exc-{qSynapse[0]}; Inh-{qSynapse[1]}")
    print(f"Synapses Dropout Rate: {dropout2}")
    print(f"Threshold: {threshold}")
    print(f"Synapse Loss Function: {synLoss.__name__}")

In [ ]:
class Pad(keras.layers.Layer):
    """flattens the temporal dimensions, order by new_order and pads with 0's to fill"""
    def __init__(self, padding, full=208, times=6, reverse=True, name="PadLayer"):
        super().__init__(name=name)
        self.full = full
        self.padding = padding
        self.times = times
        self.reverse = reverse
    
    def build(self, shape):
        pass
    
    def call(self, inputs):
                
        if self.times > 1:
            if self.reverse: new_inp = layers.Concatenate(axis=-2)([inputs, K.reverse(inputs,axes=-2)] * (self.times // 2) + [inputs] * (self.times % 2))
            else: new_inp = layers.Concatenate(axis=-2)([inputs] * self.times)
        else: new_inp = inputs
        starting_time = layers.Lambda(lambda x: self.ranInt(x))(self.padding)
        padding = self.padding[:,:, starting_time:starting_time+self.full]
        new_inp = layers.Concatenate(axis=-2)([tf.tile(padding, [tf.shape(new_inp)[0], 1, 1, 1]), new_inp])
        new_inp = K.reshape(new_inp, (tf.shape(new_inp)[0], 32, 400, 32))
        return new_inp

    def ranInt(self, x):
        return K.random_uniform((1,), 0, self.padding.shape[-2]-self.full, dtype=tf.int32)[0]#.numpy()
    
    @tf.function
    def gather(x, ind):
        return tf.gather(x+0, ind)

In [ ]:
class SliceLayer(tf.keras.layers.Layer):
    def __init__(self, start=None, end=None, name="SliceLayer"):
        super().__init__(name=name)
        self.start = start
        self.end = end

    def build(self, shape):
        self.dims = len(shape)
        pass
    
    def call(self, inputs):
        if self.start is not None:
            if self.end is not None:
                return inputs[:,:,:,self.start:self.end] if self.dims==4 else inputs[:,:,self.start:self.end]
            else:
                return inputs[:,:,:,self.start:] if self.dims==4 else inputs[:,:,self.start:]
        elif self.end is not None:
            return inputs[:,:,:,:self.end] if self.dims==4 else inputs[:,:,:self.end]

In [ ]:
def create_module(use_sigmoid=True, pruning=True, dropout1=False, dropout2=False, 
                  sigmoid_threshold=0.9, sigmoid_mult=50, to_bool=True, 
                  augment=True, optimizer=Nadam(),
                  padding=PADDING):
    
    def printModule():
        print("~*~ Visual Module ~*~")
        if augment: print("Images are augmented")
        print(f"Image Dropout Rate: {dropout1}")
        print(f"Synapse Threshold: {sigmoid_threshold}")
        print(f"Synapse Training Sigmoid Multiplication: {sigmoid_mult}")
        print(f"Synapses Dropout Rate: {dropout2}")
    
    
    printModule()
    
    inp = keras.Input(shape=(256,256,1))
    
    padding = tf.Variable(lambda: padding, trainable=False)
    x = inp
    if augment: x = data_augmentation(x)
    x = processor(x)
    if dropout1: x = layers.Dropout(dropout1)(x)
    conv_shape = (x.shape[-3], 1)
    x = Pad(padding)(x)
    x = layers.Conv2D(1278, conv_shape, strides=conv_shape, activity_regularizer=pre_synaptic_spike_regularization, name="WiringLayer")(x)
    x = layers.BatchNormalization(name="BatchNorm")(x)
    x = layers.Activation(sigmoid)(x)
    x = ToBoolLayer(threshold=sigmoid_threshold, use_sigmoid=use_sigmoid, mult=sigmoid_mult, name='preNeuronBool')(x)
    
    x = K.squeeze(x, axis=-3)

    if dropout2: x = layers.Dropout(dropout2)(x)
    x, v = L5PC_model(x)
    x = x[:, -198:, :]
#     x = L5PC_model(x)[:,-198:,:]   # run through david's model, take only the post-noise time (32 ms X 6 times)
    x = layers.MaxPooling1D(x.shape[-2], strides=x.shape[-2], name="MaxPooling")(x)
    output = layers.Flatten(name="nSpikes")(x)

    model = keras.Model(inp, [output, v])
    model.compile(optimizer=optimizer, loss={'nSpikes': MeanSquaredError()}, metrics={'nSpikes': keras.metrics.BinaryAccuracy()})
    return model

In [ ]:
def create_module_limited_syn(saccades=None, xaxis=True, use_sigmoid=True, pruning=True, dropout1=False, dropout2=False,
                        sigmoid_threshold=0.9, sigmoid_mult=15, to_bool=True, 
                        excitatory_wanted=EXCITATORY_SYNAPSES_WANTED, inhibitory_wanted=INHIBITORY_SYNAPSES_WANTED,
                        qSynapse=(tf.Variable(0.1, trainable=False), tf.Variable(0.1, trainable=False)), augment=False, 
                        optimizer=SGD(momentum=.9), conv_shape=(16,16), threshold=.2,
                        padding=PADDING, synLoss=MeanSquaredErrorSynapsesPerMS, nSynapse=True, neurons=neurons, regular_sigmoid=True, non_neg=False):
    printModule(dropout1, dropout2, sigmoid_threshold, sigmoid_mult, threshold, excitatory_wanted, inhibitory_wanted, qSynapse, augment, synLoss)
    module_name = "module_syn_"
    for i in [dropout1, dropout2, sigmoid_threshold, sigmoid_mult, threshold, excitatory_wanted, inhibitory_wanted, qSynapse, augment, synLoss]:
        module_name += str(i)
           
    inp = keras.Input(shape=(256,256,1))
    
    padding = tf.Variable(lambda: padding, trainable=False)
    x = inp
    if augment: x = data_augmentation(augment)(x)
    x = processor(x)
    if dropout1: x = layers.Dropout(dropout1)(x)
    conv_shape = (x.shape[-3], 1) if xaxis else conv_shape
    x = Pad(padding)(x)
    if non_neg: x = layers.Conv2D(2*N_EXC, conv_shape, strides=conv_shape, use_bias=True, kernel_constraint=keras.constraints.NonNeg(), name="WiringLayer")(x)#, activity_regularizer=pre_synaptic_spike_regularization)(x)
    else: x = layers.Conv2D(2*N_EXC, conv_shape, strides=conv_shape, use_bias=True, name="WiringLayer")(x)#, activity_regularizer=pre_synaptic_spike_regularization)(x)
    x = layers.BatchNormalization(name="BatchNorm")(x)
    if regular_sigmoid: x = layers.Activation(sigmoid)(x)
    x = ToBoolLayer(threshold=sigmoid_threshold, use_sigmoid=use_sigmoid, mult=sigmoid_mult, name='preNeuronBool')(x)
    
    x = K.squeeze(x, axis=-3)
        
    ExcitatorySynapses = SliceLayer(end=N_EXC, name="ExcSyns")(x)#[:, :, :N_EXC]  #SpikeProcessor(excitatory_wanted, name='nExcitatory', end=639)(x)
    InhibitorySynapses = SliceLayer(start=N_EXC, name="InhSyns")(x)#[:, :, N_EXC:]  #SpikeProcessor(inhibitory_wanted, name='nInhibitory', start=639)(x)

    if dropout2: x = layers.Dropout(dropout2)(x)
    ap, v = neurons(x)
    x = ap[:,-198:,:]   # run through david's model, take only the post-noise time (32 ms X 6 times)
    x = layers.MaxPooling1D(x.shape[-2], strides=x.shape[-2], name="MaxPooling")(x)
    x = ToBoolLayer(threshold=threshold, use_sigmoid=True, mult=1, name='postNeuronBool')(x)
    output = layers.Flatten(name="nSpikes")(x)
    
    model = keras.Model(inp, [output, ap, v])
    if not nSynapse:
        model.compile(optimizer=optimizer, loss={'nSpikes': MeanSquaredError()}, 
                      metrics={"nSpikes": keras.metrics.BinaryAccuracy()})
    else:
        model.compile(optimizer=optimizer, 
                  loss=[MeanSquaredError(), synLoss(excitatory_wanted), synLoss(inhibitory_wanted)], 
                  metrics={"nSpikes": keras.metrics.BinaryAccuracy(), "ExcSyns": MeanSynapsesPerMsMetric(), "InhSyns": MeanSynapsesPerMsMetric()},
                  loss_weights=[1.0 -qSynapse[0] - qSynapse[1], qSynapse[0], qSynapse[1]])
    return model, module_name

In [ ]:
TRAIN_MODULE = False

In [ ]:
if TRAIN_MODULE: model = create_module()

In [ ]:
if TRAIN_MODULE: model.summary()

In [ ]:
if TRAIN_MODULE: history = model.fit(train_ds, epochs=200, validation_data=valid_ds)

In [ ]:
if TRAIN_MODULE:
    
    hist = history.history

    plt.figure(figsize=(20,10))
    plt.subplot(1,2,1)
    plt.title("Loss")
    plt.plot(hist["loss"], color="k", label="train")
    plt.plot(hist["val_loss"], color="r", label="validation")
    plt.axhline(0, color="b", linestyle="--")
    plt.xlabel("epochs")
    plt.legend()

    # accuracy = "nSpikes_binary_accuracy"
    accuracy = "binary_accuracy"

    plt.subplot(1,2,2)
    plt.title("Accuracy")
    plt.plot(hist[accuracy], color="k", label=f"train (last: {str(round(hist[accuracy][-1], 2))})")
    plt.plot(hist["val_"+accuracy], color="r", label=f"validation (last: {str(round(hist['val_'+accuracy][-1], 2))})")
    plt.axhline(.5, color="b", linestyle="--", label="chance")
    plt.ylim((0,1))
    plt.xlabel("epochs")
    plt.legend()

    plt.show()

In [ ]:
if TRAIN_MODULE: model.evaluate(test_ds)

# Plot

In [ ]:
def plot_examples(model, startFrom=200, plot_cycle=False, plot_spikes=True, plot_start=None, write_last=False, weights_start=None, preNeuron="NeuronInput", spikeTrain="SpikeTrain", postNeuron="nSpikes"):
    plt.figure(figsize=(200, 250))
    how_many = 10
    for img, label in train_ds.take(1):
#         inputs = K.function(model.input, model.get_layer(bool_layers[0]).output)([img])
        inputs = K.function(model.input, model.get_layer(preNeuron).output)([img])
        outputs = K.function(model.input, model.get_layer(spikeTrain).output)([img])
        if write_last: nAP = K.function(model.input, model.get_layer(postNeuron).output)([img])

        for i in range(how_many):
            plt.subplot(how_many, 3, 3*i+1)
            plt.imshow(img[i,:,:,0]/255, cmap='gray')
            plt.title(LABELS[label[i]], fontdict={'fontsize':200})
            plt.axis("off")
            plt.subplot(how_many, 3, 3*i+2)
            if len(inputs[i].shape)==3:
                curr_input = np.squeeze(inputs[i], axis=-3)[startFrom:]
            else:
                curr_input = inputs[i][startFrom:]
            num_of_synapses = np.sum(curr_input, axis=-1)
            mean_synapses = round(num_of_synapses.mean(), 2)
            std_synapses = round(num_of_synapses.std(), 2)
            plt.title(f"\u03BC: {str(mean_synapses)},  \u03C3: {str(std_synapses)}", fontdict={'fontsize':200})
            plt.imshow(curr_input[-32:,:] if plot_cycle else curr_input, cmap='binary', vmin=0, vmax=1)
            plt.axis("off")
            plt.subplot(how_many, 3, 3*i+3)
            curr_output = outputs[i]            
            spikes = []
            curr_index = startFrom
            while True:
                start = np.where(curr_output[curr_index:] > 0.25)[0]
                if not start.shape[0]: break
                start = curr_index+start[0]
                end = np.where(curr_output[start:] < 0.1)[0]
                if not end.shape[0]: break
                end = end[0] + start
                spikes.append((start,end))
                curr_index = end + 1
            plt.title((f"Score: {str(round(nAP[i][0],2))}," if write_last else "") +f"\u03A3: {str(round(curr_output.sum(),2))}" + (f", spikes:{spikes}" if plot_spikes else ""), fontdict={'fontsize':150})
            plt.plot(curr_output, linewidth=5)
            if weights_start is not None:
                plt.plot(np.arange(weights_start, 400), model.get_layer("nSpikes").get_weights()[0][:,0], color='r')
            plt.ylim(0,1)
            if plot_start: plt.axvline(plot_start, color='g', linestyle=':', linewidth=10.)
            plt.axis('on')
        break

In [ ]:
def plot_statistics(model, starting_time=0, cycle_time=32, preNeuron="NeuronInput", spikeTrain="SpikeTrain"):
    
    all_inputs = []
    n_synapses = []
    synapses_mu = []
    synapses_std = []
    synapses_max = []
    synapses_min = []
#     sum_output = []
    all_outputs = []
#     excitatory_synapses = []
#     inhibitory_synapses = []
    label_dct = {}
    synapses_mean_per_label = {}
    
    for img, label in train_ds.take(25):
        inputs = K.function(model.input, model.get_layer(preNeuron).output)([img])
        outputs = K.function(model.input, model.get_layer(spikeTrain).output)([img])
        all_inputs.append(inputs)
        labels = [LABELS[lbl] for lbl in label]
        for j in range(len(inputs)):
            if len(inputs[j].shape)==3:
                curr_input = np.squeeze(inputs[j], axis=-3)
            else:
                curr_input = inputs[j]
            
            num_of_synapses = np.sum(curr_input[starting_time:starting_time+cycle_time, :N_EXC], axis=-1)
#             excitatory_synapses.append(np.sum(curr_input[:, :N_EXC], axis=-1).mean())
#             inhibitory_synapses.append(np.sum(curr_input[:, N_EXC:], axis=-1).mean())
            n_synapses.append(num_of_synapses)
            synapses_mu.append(num_of_synapses.mean())
            synapses_std.append(num_of_synapses.std())
            synapses_max.append(num_of_synapses.max())
            synapses_min.append(num_of_synapses.min())
            curr_output = outputs[j]
#             sum_output.append(curr_output.sum())
            all_outputs.append(curr_output)
            if labels[j] not in label_dct:
                label_dct[labels[j]] = []
                synapses_mean_per_label[labels[j]] = {"Sum": [], "Ex":[], "Inh":[]}
            label_dct[labels[j]].append(curr_output)
            synapses_mean_per_label[labels[j]]["Sum"].append(np.sum(curr_input[starting_time:starting_time+cycle_time, :N_EXC], axis=-1).mean())
            synapses_mean_per_label[labels[j]]["Ex"].append(np.sum(curr_input[starting_time:starting_time+cycle_time, :N_EXC], axis=-1).mean())
            synapses_mean_per_label[labels[j]]["Inh"].append(np.sum(curr_input[starting_time:starting_time+cycle_time, N_EXC:], axis=-1).mean())

    mean_dct = {}
    for lbl in label_dct.keys():
        mean_dct[lbl] = np.stack(label_dct[lbl]).mean(axis=0)

    plt.figure(figsize=(15,10))

    plt.subplot(3,3,1)
    plt.title('input values')
    plt.hist(np.array(all_inputs).ravel())

    plt.subplot(3,3,2)
    plt.title('mean of sum synapses')
    plt.hist(synapses_mu)

    plt.subplot(3,3,3)
    plt.title('std of sum synapses')
    plt.hist(synapses_std)

    plt.subplot(3,3,4)
    plt.title('max sum synapses')
    plt.hist(synapses_max)

    plt.subplot(3,3,5)
    plt.title('min sum synapses')
    plt.hist(synapses_min)

    plt.subplot(3,3,6)
    plt.title(f'mean output of neuron (AP)')
    for lbl, value in mean_dct.items():
        plt.plot(value, label=lbl, linewidth=2.)
    plt.ylim(0,1)
    plt.legend()

    plt.subplot(3,3,7)
    plt.title('Excitatory Synapses Mean per Label')
    plt.hist([synapses_mean_per_label[lbl]["Ex"] for lbl in synapses_mean_per_label.keys()], label=list(synapses_mean_per_label.keys()))
    plt.legend()
    plt.xlim((0,15))


    plt.subplot(3,3,8)
    plt.title('Inhibitory Synapses Mean per Label')
    plt.hist([synapses_mean_per_label[lbl]["Inh"] for lbl in synapses_mean_per_label.keys()], label=list(synapses_mean_per_label.keys()))
    plt.legend()
    plt.xlim((0,15))

    
    plt.subplot(3,3,9)
    plt.title('All Synapses Mean per Label')
    plt.hist([synapses_mean_per_label[lbl]["Sum"] for lbl in synapses_mean_per_label.keys()], label=list(synapses_mean_per_label.keys()))
    plt.legend()

    plt.show()

In [ ]:
if TRAIN_MODULE: 
    import re
    moduleLayer = r"tf.__operators__.getitem_\d+"
    selectedLayer = None
    for layer in model.layers:
        if bool(re.search(moduleLayer, layer.name)):
            selectedLayer = layer.name
            break
        else: print(layer.name)
    print(selectedLayer)

In [ ]:
if TRAIN_MODULE: plot_examples(model, startFrom=0, plot_cycle=False, plot_spikes=False, plot_start=None, write_last=True, preNeuron="preNeuronBool", spikeTrain=selectedLayer)

In [ ]:
if TRAIN_MODULE: plot_statistics(model, 200 + 200%32, preNeuron="preNeuronBool", spikeTrain=selectedLayer)

In [ ]:
if TRAIN_MODULE: 
    plt.figure(figsize=(5,15))
    weights = model.get_layer("WiringLayer").get_weights()[0][0]
    plt.suptitle(f"Weights Examples (8 synapses out of {2*N_EXC})")
    for i in range(8):
        for j in range(4):
            plt.subplot(8, 4, 4*i+j+1)
            x = np.zeros((32,8))
            for l in range(32//4):
                x[:,l] = weights[:,l*4+j,i]
            plt.imshow(x, vmin=tf.reduce_min(weights), vmax=tf.reduce_max(weights))
            if not i:
                plt.title(r"$\Theta={}\pi$".format(thetas[j]/np.pi))
            if not j:
                plt.ylabel(f"Syn {i}\nyaxis pixels")
            if i == 7:
                plt.xlabel(f"filter")
            plt.xticks([])
            plt.yticks([])

In [ ]:
if TRAIN_MODULE: 
    plt.plot(weights.mean(axis=(0,2)), label="mean")
    plt.plot(np.abs(weights).mean(axis=(0,2)), label="abs mean")
    plt.title("Mean and Abs Mean of Synapse Weights by Channel (after processing)")
    plt.legend()
    plt.show()

In [ ]:
if TRAIN_MODULE:
    plt.figure(figsize=(20,20))

    plt.subplot(2,2,1)
    plt.title(r"Exc Synapses Weights (std by Synapse)")
    plt.hist(weights[:,:,:N_EXC].std(axis=(0,1)))

    plt.subplot(2,2,3)
    plt.title(r"Exc Synapses Weights (sum by Synapse)")
    plt.hist(weights[:,:,:N_EXC].sum(axis=(0,1)))

    plt.subplot(2,2,2)
    plt.title(r"Inh Synapses Weights (std by Synapse)")
    plt.hist(weights[:,:,N_EXC:].std(axis=(0,1)))

    plt.subplot(2,2,4)
    plt.title(r"Inh Synapses Weights (sum by Synapse)")
    plt.hist(weights[:,:,N_EXC:].sum(axis=(0,1)))

    plt.show()

In [ ]:
if TRAIN_MODULE: 
    # plt.figure(figsize=(10,20))
    weights = model.get_layer("WiringLayer").get_weights()[0][0]
    print(f"min: {weights.min()}, max: {weights.max()}\nmean: {weights.mean()}, sd: {weights.std()}\n")
    exc = weights[:,:,:N_EXC]
    inh = weights[:,:,N_EXC:]
    amountExc = [(exc>0.01).sum(axis=(0,1))]
    amountInh = [(inh>0.01).sum(axis=(0,1))]

    plt.subplot(1,2,1)
    plt.hist(amountExc)
    plt.title("Exc Synapses per Pixel")
    plt.ylabel("Synapses per Pixel")

    plt.subplot(1,2,2)
    plt.hist(amountInh)
    plt.title("Inh Synapses per Pixel")

    plt.show()

# Convert Images to 400X1278 matrices

# Save Weights

In [ ]:
if TRAIN_MODULE: serial = 0

In [ ]:
if TRAIN_MODULE: 
    serial += 1
    with open(f"weights{serial}.npy", 'wb') as f:
        np.save(f, model.get_layer("WiringLayer").get_weights()[0])
    with open(f"bias{serial}.npy", 'wb') as f:
        np.save(f, model.get_layer("WiringLayer").get_weights()[1])
    with open(f"batchnorm{serial}.npy", 'wb') as f:
        np.save(f, model.get_layer("BatchNorm").get_weights())

In [ ]:
if TRAIN_MODULE: model.save(f"./model{serial}")

# Load Pretrained Module

In [ ]:
create_limited_syn = True

In [ ]:
if not create_limited_syn: direc = "../input/visual-module-76-variance-loss/"

In [ ]:
if not create_limited_syn: 
    module = create_module(sigmoid_threshold=0.9, sigmoid_mult=75)
    weights = []
    with open(direc+"weights1.npy", 'rb') as f: weights.append(np.load(f))
    with open(direc+"bias1.npy", 'rb') as f: weights.append(np.load(f))
    module.get_layer("WiringLayer").set_weights(weights)
    with open(direc+"batchnorm1.npy", 'rb') as f: module.get_layer("BatchNorm").set_weights(np.load(f))

In [ ]:
if not create_limited_syn: module.evaluate(test_ds)

# The Same with Limited Synapses

In [ ]:
direc = "../input/threshold-02-no-synapses-76-validation-76-test"

In [ ]:
if create_limited_syn:
    module_syn, module_name = create_module_limited_syn(threshold=.2, non_neg=False, nSynapse=False, dropout1=0.6, dropout2=0.4, use_sigmoid=True, regular_sigmoid=True, sigmoid_mult=50, sigmoid_threshold=.9, augment=False, optimizer=Nadam(), qSynapse=(.1, .1), synLoss=MeanSquaredErrorSynapsesPerMS)
    weights = []
    with open(direc+"/module_syn_0.60.40.9500.286(0.1 0.1)1MeanSquaredErrorSynapsesPerMS_weights.npy", 'rb') as f: weights.append(np.load(f))
    with open(direc+"/module_syn_0.60.40.9500.286(0.1 0.1)1MeanSquaredErrorSynapsesPerMS_bias.npy", 'rb') as f: weights.append(np.load(f))
    module_syn.get_layer("WiringLayer").set_weights(weights)
    with open(direc+"/module_syn_0.60.40.9500.286(0.1 0.1)1MeanSquaredErrorSynapsesPerMS_batchnorm.npy", 'rb') as f: module_syn.get_layer("BatchNorm").set_weights(np.load(f))
    module_syn.evaluate(test_ds)

# Convert Dataset Using Pretrained Module

In [ ]:
module_to_use = module_syn if create_limited_syn else module

In [ ]:
import re
moduleLayer = r"tf.__operators__.getitem_\d+"
selectedLayer = None
for layer in module_to_use.layers:
    if bool(re.search(moduleLayer, layer.name)):
        selectedLayer = layer.name
        break
    else: print(layer.name)
print(selectedLayer)

In [ ]:
preNeuron="preNeuronBool"
spikeTrain=selectedLayer
postNeuron="nSpikes"

In [ ]:
os.makedirs('../working/cats-dogs-matrices')

In [ ]:
how_many_to_transform_per_label = None  # set to zero or None to convert all the dataset
datasets_to_convert = [(train_ds, "training"), (valid_ds, "validation"), (test_ds, "test")]
datasets_to_convert = datasets_to_convert[1:]  # change 

In [ ]:
import shutil
def zip_and_delete(directory, zip_name, to_zip=True):
    if not to_zip: return
    shutil.make_archive(zip_name, 'zip', directory)
    print(f'Done Zipping! Check for {zip_name}.')
    shutil.rmtree(directory)
    print('Done erasing photos!')

In [ ]:
stop = False
# how_much_wanted = 250
for ds, dir_name in datasets_to_convert:
    dct = {lbl: 0 for lbl in LABELS}
    os.makedirs('../working/cats-dogs-matrices/'+dir_name)
    for lbl in LABELS:
        os.makedirs(f'../working/cats-dogs-matrices/{dir_name}/{lbl}')
    
    for batchN, (img, label) in enumerate(ds):
#         if batchN >= how_much_wanted: break
        inputs = K.function(module_to_use.input, module_to_use.get_layer(preNeuron).output)([img])
        outputs = K.function(module_to_use.input, module_to_use.get_layer(spikeTrain).output)([img])
        nAP = K.function(module_to_use.input, module_to_use.get_layer(postNeuron).output)([img])
        _, ap, somaV = module_to_use(img)

        if how_many_to_transform_per_label and sum(dct.values()) >= 2*how_many_to_transform_per_label: break

        for i in range(inputs.shape[0]):
            im = img[i]
            lbl = LABELS[label[i]]
            inp = inputs[i]
            out = outputs[i]
            pred = nAP[i][0]
            v = somaV[i,:,0]
            predLbl = LABELS[int(round(pred, 0))]
            if how_many_to_transform_per_label and dct[lbl] >= how_many_to_transform_per_label: continue
            else:
                curDir = f'../working/cats-dogs-matrices/{dir_name}/{lbl}/{dct[lbl]}'
                os.makedirs(curDir)
                with open(curDir+"/image.npy", 'wb') as f: np.save(f, im)
                with open(curDir+"/matrix.npy", 'wb') as f: np.save(f, inp)
                with open(curDir+"/spikePrediction.npy", 'wb') as f: np.save(f, out)
                with open(curDir+"/voltagePrediction.npy", 'wb') as f: np.save(f, v)
                with open(curDir+'/predDict.p', 'wb') as f: pickle.dump({"correct": lbl, "predictedLbl": predLbl, "predictSpike": pred}, f, protocol=pickle.HIGHEST_PROTOCOL)
                dct[lbl] += 1
                print(curDir)
    zip_and_delete('../working/cats-dogs-matrices/'+dir_name, '../working/cats-dogs-matrices/'+dir_name+"zip", to_zip=True)